In [1]:
import pandas as pd
from typing import List, Tuple
import re

def tokenize_tamil_text(text: str) -> List[str]:
    """
    தமிழ் உரையைச் சொற்களாகப் பிரிக்கும் செயல்பாடு
    """
    # சொற்களைப் பிரிப்பதற்கான அடிப்படை எல்லைகள்
    word_boundaries = r'[.,!? ]+'

    # உரையைச் சொற்களாகப் பிரித்தல்
    words = re.split(word_boundaries, text)

    # வெற்று சொற்களை நீக்குதல்
    words = [word.strip() for word in words if word.strip()]

    return words

def create_question_prompt(question: str) -> str:
    """
    கேள்வியை வினவல் வடிவமாக மாற்றும் செயல்பாடு
    """
    return f"கேள்வி: {question}\nபதில் தருக: "

def evaluate_answer(expected: str, actual: str) -> Tuple[float, str]:
    """
    பதிலின் துல்லியத்தை மதிப்பிடும் செயல்பாடு
    """
    expected_words = set(tokenize_tamil_text(expected.lower()))
    actual_words = set(tokenize_tamil_text(actual.lower()))

    # பொதுவான சொற்களின் எண்ணிக்கையைக் கணக்கிடுதல்
    common_words = expected_words.intersection(actual_words)

    # துல்லியத்தை கணக்கிடுதல்
    if len(expected_words) == 0:
        accuracy = 0
    else:
        accuracy = len(common_words) / len(expected_words) * 100

    # தரப்புள்ளி வழங்குதல்
    if accuracy >= 90:
        grade = 'A'
    elif accuracy >= 80:
        grade = 'B'
    elif accuracy >= 70:
        grade = 'C'
    elif accuracy >= 60:
        grade = 'D'
    else:
        grade = 'F'

    return accuracy, grade

def analyze_qa_dataset(file_path: str):
    """
    முழு dataset-ஐ பகுப்பாய்வு செய்யும் முதன்மை செயல்பாடு
    """
    # Dataset-ஐ படித்தல்
    df = pd.read_csv(file_path)

    results = []
    for index, row in df.iterrows():
        question = row['Prompt']
        expected_answer = row['Response']

        # சொற்களைப் பிரித்தல்
        question_tokens = tokenize_tamil_text(question)
        answer_tokens = tokenize_tamil_text(expected_answer)

        # வினவல் உருவாக்குதல்
        prompt = create_question_prompt(question)

        # பதிலை மதிப்பீடு செய்தல்
        accuracy, grade = evaluate_answer(expected_answer, expected_answer)  # இங்கே உண்மையான பதிலை ஒப்பிடலாம்

        results.append({
            'Question': question,
            'Question_Tokens': question_tokens,
            'Answer': expected_answer,
            'Answer_Tokens': answer_tokens,
            'Prompt': prompt,
            'Accuracy': accuracy,
            'Grade': grade
        })

    return pd.DataFrame(results)

# பயன்பாட்டு உதாரணம்
if __name__ == "__main__":
    # Dataset கோப்பு பாதை
    file_path = "tholkaappiyam_dataset.csv"

    # பகுப்பாய்வு செய்தல்
    results_df = analyze_qa_dataset(file_path)

    # முடிவுகளைக் காட்டுதல்
    print("\nபகுப்பாய்வு முடிவுகள்:")
    print("-" * 50)
    for index, row in results_df.iterrows():
        print(f"\nவினா {index + 1}:")
        print(f"கேள்வி: {row['Question']}")
        print(f"சொற்கள்: {', '.join(row['Question_Tokens'])}")
        print(f"வினவல்:\n{row['Prompt']}")
        print(f"எதிர்பார்க்கப்படும் பதில்: {row['Answer']}")
        print(f"பதில் சொற்கள்: {', '.join(row['Answer_Tokens'])}")
        print(f"துல்லியம்: {row['Accuracy']:.2f}%")
        print(f"தரம்: {row['Grade']}")
        print("-" * 50)


பகுப்பாய்வு முடிவுகள்:
--------------------------------------------------

வினா 1:
கேள்வி: தொகைமரபு நூற்பா 1 எழுதித் தரவும்.
சொற்கள்: தொகைமரபு, நூற்பா, 1, எழுதித், தரவும்
வினவல்:
கேள்வி: தொகைமரபு நூற்பா 1 எழுதித் தரவும்.
பதில் தருக: 
எதிர்பார்க்கப்படும் பதில்: க ச த ப முதலிய மொழிமேல் தோன்றும்
மெல்லெழுத்து இயற்கை சொல்லிய முறையான்
ங ஞ ந ம என்னும் ஒற்று ஆகும்மே
அன்ன மரபின் மொழிவயினான. 1
பதில் சொற்கள்: க, ச, த, ப, முதலிய, மொழிமேல், தோன்றும்
மெல்லெழுத்து, இயற்கை, சொல்லிய, முறையான்
ங, ஞ, ந, ம, என்னும், ஒற்று, ஆகும்மே
அன்ன, மரபின், மொழிவயினான, 1
துல்லியம்: 100.00%
தரம்: A
--------------------------------------------------

வினா 2:
கேள்வி: தொகைமரபு நூற்பா 2 எழுதித் தரவும்.
சொற்கள்: தொகைமரபு, நூற்பா, 2, எழுதித், தரவும்
வினவல்:
கேள்வி: தொகைமரபு நூற்பா 2 எழுதித் தரவும்.
பதில் தருக: 
எதிர்பார்க்கப்படும் பதில்: ஞ ந ம ய வ எனும் முதல் ஆகு மொழியும்
உயிர் முதல் ஆகிய மொழியும் உளப்பட
அன்றி அனைத்தும் எல்லா வழியும்
நின்ற சொல் முன் இயல்பு ஆகும்மே. 2
பதில் சொற்கள்: ஞ, ந, ம, ய, வ, எனும், முதல், ஆகு, மொழியும்

In [2]:
import pandas as pd
from typing import List, Tuple, Dict
import re
from difflib import SequenceMatcher
import random

class TamilQASystem:
    def __init__(self, file_path: str):
        """
        தமிழ் கேள்வி-பதில் அமைப்பை துவக்குதல்
        """
        self.df = pd.read_csv(file_path)
        self.knowledge_base = self._build_knowledge_base()

    def _build_knowledge_base(self) -> Dict[str, List[str]]:
        """
        தரவிலிருந்து அறிவுத்தளத்தை உருவாக்குதல்
        """
        knowledge_base = {}
        for _, row in self.df.iterrows():
            tokens = self.tokenize_tamil_text(row['Prompt'].lower())
            for token in tokens:
                if token not in knowledge_base:
                    knowledge_base[token] = []
                knowledge_base[token].append(row['Response'])
        return knowledge_base

    @staticmethod
    def tokenize_tamil_text(text: str) -> List[str]:
        """
        தமிழ் உரையைச் சொற்களாகப் பிரித்தல்
        """
        word_boundaries = r'[.,!? ]+'
        words = re.split(word_boundaries, text)
        words = [word.strip() for word in words if word.strip()]
        return words

    def create_question_prompt(self, question: str) -> str:
        """
        கேள்வியை வினவல் வடிவமாக மாற்றுதல்
        """
        return f"கேள்வி: {question}\nபதில் தருக: "

    def evaluate_answer(self, expected: str, actual: str) -> Tuple[float, str]:
        """
        பதிலின் துல்லியத்தை மதிப்பிடுதல்
        """
        expected_words = set(self.tokenize_tamil_text(expected.lower()))
        actual_words = set(self.tokenize_tamil_text(actual.lower()))

        common_words = expected_words.intersection(actual_words)

        if len(expected_words) == 0:
            accuracy = 0
        else:
            accuracy = len(common_words) / len(expected_words) * 100

        if accuracy >= 90:
            grade = 'A'
        elif accuracy >= 80:
            grade = 'B'
        elif accuracy >= 70:
            grade = 'C'
        elif accuracy >= 60:
            grade = 'D'
        else:
            grade = 'F'

        return accuracy, grade

    def find_similar_questions(self, question: str, threshold: float = 0.6) -> List[Tuple[str, float]]:
        """
        கொடுக்கப்பட்ட கேள்விக்கு ஒத்த கேள்விகளைக் கண்டறிதல்
        """
        similar_questions = []
        for _, row in self.df.iterrows():
            similarity = SequenceMatcher(None, question.lower(),
                                      row['Prompt'].lower()).ratio()
            if similarity >= threshold:
                similar_questions.append((row['Prompt'], similarity))

        return sorted(similar_questions, key=lambda x: x[1], reverse=True)

    def generate_new_answer(self, question: str) -> Tuple[str, List[str]]:
        """
        புதிய கேள்விக்கு பதிலை உருவாக்குதல்
        """
        # ஒத்த கேள்விகளைக் கண்டறிதல்
        similar_questions = self.find_similar_questions(question)

        if not similar_questions:
            return "மன்னிக்கவும், இந்தக் கேள்விக்கு பதில் கண்டறிய முடியவில்லை.", []

        # பதில் உருவாக்கத்திற்கான தரவுகளைச் சேகரித்தல்
        relevant_responses = []
        for q, _ in similar_questions[:3]:  # மிகவும் பொருத்தமான 3 கேள்விகளை எடுத்துக்கொள்ளுதல்
            relevant_responses.extend(
                self.df[self.df['Prompt'] == q]['Response'].tolist()
            )

        # பதிலை உருவாக்குதல்
        if len(relevant_responses) == 1:
            return relevant_responses[0], similar_questions
        else:
            # பல பதில்களிலிருந்து புதிய பதிலை உருவாக்குதல்
            response_parts = []
            for response in relevant_responses:
                sentences = re.split('[.!?]+', response)
                sentences = [s.strip() for s in sentences if s.strip()]
                if sentences:
                    response_parts.append(random.choice(sentences))

            if response_parts:
                new_response = '. '.join(response_parts) + '.'
                return new_response, similar_questions
            else:
                return "மன்னிக்கவும், பொருத்தமான பதில் உருவாக்க முடியவில்லை.", []

    def analyze_dataset(self):
        """
        முழு dataset-ஐ பகுப்பாய்வு செய்தல்
        """
        results = []
        for index, row in self.df.iterrows():
            question = row['Prompt']
            expected_answer = row['Response']

            question_tokens = self.tokenize_tamil_text(question)
            answer_tokens = self.tokenize_tamil_text(expected_answer)

            prompt = self.create_question_prompt(question)
            accuracy, grade = self.evaluate_answer(expected_answer, expected_answer)

            results.append({
                'Question': question,
                'Question_Tokens': question_tokens,
                'Answer': expected_answer,
                'Answer_Tokens': answer_tokens,
                'Prompt': prompt,
                'Accuracy': accuracy,
                'Grade': grade
            })

        return pd.DataFrame(results)

def main():
    """
    செயல்முறை விளக்க உதாரணம்
    """
    # அமைப்பை துவக்குதல்
    qa_system = TamilQASystem("tholkaappiyam_dataset.csv")

    # தரவுத்தள பகுப்பாய்வு
    results_df = qa_system.analyze_dataset()

    # பகுப்பாய்வு முடிவுகளைக் காட்டுதல்
    print("\nதரவுத்தள பகுப்பாய்வு முடிவுகள்:")
    print("-" * 50)
    for index, row in results_df.iterrows():
        print(f"\nவினா {index + 1}:")
        print(f"கேள்வி: {row['Question']}")
        print(f"சொற்கள்: {', '.join(row['Question_Tokens'])}")
        print(f"வினவல்:\n{row['Prompt']}")
        print(f"பதில்: {row['Answer']}")
        print(f"பதில் சொற்கள்: {', '.join(row['Answer_Tokens'])}")
        print(f"துல்லியம்: {row['Accuracy']:.2f}%")
        print(f"தரம்: {row['Grade']}")
        print("-" * 50)

    # புதிய கேள்விக்கு பதில் உருவாக்குதல்
    test_question = "எழுத்துக்களின் வகைகள் யாவை?"
    generated_answer, similar_questions = qa_system.generate_new_answer(test_question)

    print("\nபுதிய கேள்வி பதில் உருவாக்கம்:")
    print("-" * 50)
    print(f"கேள்வி: {test_question}")
    print("\nஒத்த கேள்விகள்:")
    for q, similarity in similar_questions:
        print(f"- {q} (ஒற்றுமை: {similarity:.2f})")
    print(f"\nஉருவாக்கப்பட்ட பதில்: {generated_answer}")

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
பதில் சொற்கள்: சூ, 151, :	
உயிரீ, றாகிய, முன்னிலைக், கிளவியும்

புள்ளி, யிறுதி, முன்னிலைக், கிளவியும்

இயல்பா, குநவும், உறழ்பா, குநவுமென்று

ஆயீ, ரியல, வல்லெழுத்து, வரினே

(9), க-து, :	
உயிரீறும், புள்ளியீறுமாகிய, முன்னிலை, வினைச்சொற்கள்
வன்கணத்தொடு, புணருமாறு, கூறுகின்றது, பொருள்:உயிரெழுத்து, ஈறாகி, நிற்கும், முன்னிலைவினைச்
சொற்களும், புள்ளி, எழுத்து, ஈறாகி, நிற்கும், முன்னிலை, வினைச், சொற்களும், வல்லெழுத்தை, முதலாக, உடைய, சொற்கள்வரின், இயல்பாக, முடிவனவும்
உறழ்ந்து, முடிவனவும், என, இரண்டடியல்பினையுடையவாகும், அடுத்துவரும், சூத்திரத்துள், வினையீறுகளை, விதந்து, ‘‘முற்றத், தோன்றா
முன்னிலை, மொழிக்கே’’, எனக், கூறுதலின், ஈண்டு, முன்னிலைக், கிளவி
என்றது, முன்னிலை, வினைச்சொல்லை, என்பது, பெறப்படும், முன்னிலை, வினைகள், பால்காட்டும், இறுதி, இடைச்சொல், பெற்று
வருவனவும், முதனிலைத், தொழிற்பெயர்கள், எடுத்தலோசைான்
ஈற்றிடைச், சொல், குன்றி, வருவனவுமென, இரு, நிலைமையவாக, வரும், முன்னிலை, வினையீறுகளாவன:, இ, ஐ, ஆய், இர், ஈர், மின், என
வினையியலுள், ஓதப், பெற